In [14]:
from cloudsafe import *
from utils.folium_utils import get_geojson_grid
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
database = 'here-api/main.db'
conn = create_connection(database)
departures = select_all_departures(conn)
stations = select_all_stations(conn)
bus_stops = select_all_bus_stops()

In [20]:
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)

def get_coords(row):
    if row['coordinates_here']:
        coords = row['coordinates_here'].split(',') 
        return coords[0], coords[1]
    return None, None

stations[['latitude', 'longitude']] = stations.apply(get_coords, axis=1, result_type="expand")

# for index, row in stations.iterrows():
#     folium.CircleMarker([row['latitude'], row['longitude']],
#                         radius=15,
#                         popup=row['name_here'],
#                         fill_color="#3db7e4", # divvy color
#                        ).add_to(m)



In [21]:
# convert to (n, 2) nd-array format for heatmap
stationArr = stations[['latitude', 'longitude']].to_numpy()
# stationArr
# plot heatmap
# m.add_children(HeatMap(stationArr, radius=15))
# m

In [48]:
#choropleth map
frequencies = stations.name_overpass.value_counts()

# final = pd.DataFrame(columns=['station_name', 'longitude', 'latitude', 'bus_count'])
# stations
rows = []
for station in frequencies.keys():
    first = stations.loc[stations['name_overpass'] == station].iloc[0]
    rows.append({'station_name': station, 'longitude': first.longitude, 'latitude': first.latitude, 'bus_count': frequencies.get(station)})
    
final = pd.DataFrame(rows)
final['longitude'] = final['longitude'].astype(float)
final['latitude'] = final['latitude'].astype(float)
# final

# max_latitude = final.loc[final['latitude'].astype(float).idxmax()]
# min_latitude = final.loc[final['latitude'].astype(float).idxmin()]

# max_longitude = final.loc[final['longitude'].astype(float).idxmax()]
# min_longitude = final.loc[final['longitude'].astype(float).idxmin()]

top_right = [final['latitude'].max(), final['longitude'].max()]

bottom_left = [final['latitude'].min(), final['longitude'].min()]

grid = get_geojson_grid(top_right, bottom_left)

counts = []

for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]

    items = final.loc[(final['latitude'] <= upper_right[1]) & ( final['longitude'] <= upper_right[0]) & (final['longitude'] >= lower_left[0]) & (final['latitude'] >= lower_left[1]) ]
    counts.append(len(items))
    
max_count = max(counts)


for i, geo_json in enumerate(grid):

    color = plt.cm.Reds(counts[i] / max_count)
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })

    m.add_child(gj)
m

# top_left = final['latitude'].astype(float)
# top_left.max()

# get_geojson_grid()
# for index, row in final.iterrows():
#     folium.CircleMarker([row['latitude'], row['longitude']],
#                         radius=15,
#                         popup=row['station_name'],
#                         fill_color="#3db7e4", # divvy color
#                        ).add_to(m)
# m

0
0
0
0
8
0
0
12
16
11
9
14
4
21
18
27
21
19
6
12
3
23
21
7
10
15
19
21
22
4
11
5
11
16
14
2
